In [1]:
from google.colab import drive
drive.mount('/content/drive')

# replace this path with your data folder location
data_path = "/content/drive/My Drive/GAIN/nasa space challenge 2019/data/"

# replace this path with your root path
root_path = "/content/drive/My Drive/GAIN/nasa space challenge 2019/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## UCI - Combined Cycle Power Plant Data Set

### Abstract: 
The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the plant was set to work with full load.

### Data Set Information:

The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. Features consist of hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V) to predict the net hourly electrical energy output (PE) of the plant.

https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant

In [2]:
import pandas as pd
# Data generation
original_dataset = pd.read_csv(data_path+"power_plant.csv", sep=',')
original_dataset.head(5)

,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43


In [0]:
# Modification to use another normalization techniques
# Normalization (0 to 1)  
from sklearn.preprocessing import MinMaxScaler, RobustScaler   
scaler = MinMaxScaler()
original_dataset[original_dataset.columns] = scaler.fit_transform(original_dataset[original_dataset.columns])

In [4]:
original_dataset.describe()

,AT,V,AP,RH,PE
count,9568.000000,9568.000000,9568.000000,9568.000000,9568.000000
mean,0.505417,0.515050,0.504060,0.640067,0.451722
std,0.211118,0.226119,0.146963,0.195714,0.226053
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.331445,0.291459,0.401138,0.506267,0.258146
50%,0.525071,0.475445,0.496164,0.662399,0.414437
75%,0.677337,0.732740,0.603069,0.794504,0.638013
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
# split original dataset (train,test)
# test_set wont have missing values for realistic algorithm comparison

from sklearn.model_selection import train_test_split

# we will use full_set to compare MSE between [original dataset vs GAIN generated dataset]
full_set, test_set = train_test_split(original_dataset, test_size=0.50, random_state=2)

# we will take 80% of dataset to compare MSE between [GAIN generated dataset (full_set) vs partial_set (Full set at 20% missing)] 
partial_set, _ = train_test_split(full_set,test_size = 0.2, random_state=1)


Data = full_set.to_numpy()

## Generative Adversarial Imputation Networks (GAIN)

Written by Jinsung Yoon
Date: Jan 29th 2019
Generative Adversarial Imputation Networks (GAIN) Implementation on Spam 
Dataset Reference: J. Yoon, J. Jordon, M. van der Schaar, "GAIN: Missing Data Imputation using Generative Adversarial Nets," ICML, 2018.

In [6]:
'''
Written by Jinsung Yoon
Date: Jan 29th 2019
Generative Adversarial Imputation Networks (GAIN) Implementation on Spam Dataset
Reference: J. Yoon, J. Jordon, M. van der Schaar, "GAIN: Missing Data Imputation using Generative Adversarial Nets," ICML, 2018.
Paper Link: http://medianetlab.ee.ucla.edu/papers/ICML_GAIN.pdf
Appendix Link: http://medianetlab.ee.ucla.edu/papers/ICML_GAIN_Supp.pdf
Contact: jsyoon0823@g.ucla.edu
'''

#%% Packages
import tensorflow as tf
import numpy as np
from tqdm import tqdm

#%% System Parameters
# 1. Mini batch size
mb_size = 128
# 2. Missing rate
p_miss = 0.2
# 3. Hint rate
p_hint = 0.9
# 4. Loss Hyperparameters
alpha = 10
# 5. Train Rate
train_rate = 0.8

#%% Data

# Data generation
#Data = np.loadtxt(data_path+"power_plant.csv", delimiter=",",skiprows=1)

# Parameters
No = len(Data)
Dim = len(Data[0,:])

# Hidden state dimensions
H_Dim1 = Dim
H_Dim2 = Dim

# Normalization (0 to 1)
#Min_Val = np.zeros(Dim)
#Max_Val = np.zeros(Dim)

#for i in range(Dim):
#    Min_Val[i] = np.min(Data[:,i])
#    Data[:,i] = Data[:,i] - np.min(Data[:,i])
#    Max_Val[i] = np.max(Data[:,i])
#    Data[:,i] = Data[:,i] / (np.max(Data[:,i]) + 1e-6)   
 

#%% Missing introducing
p_miss_vec = p_miss * np.ones((Dim,1)) 
   
Missing = np.zeros((No,Dim))

for i in range(Dim):
    A = np.random.uniform(0., 1., size = [len(Data),])
    B = A > p_miss_vec[i]
    Missing[:,i] = 1.*B

    
#%% Train Test Division    
   
idx = np.random.permutation(No)

Train_No = int(No * train_rate)
Test_No = No - Train_No
    
# Train / Test Features
trainX = Data[idx[:Train_No],:]
testX = Data[idx[Train_No:],:]

# Train / Test Missing Indicators
trainM = Missing[idx[:Train_No],:]
testM = Missing[idx[Train_No:],:]

#%% Necessary Functions

# 1. Xavier Initialization Definition
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape = size, stddev = xavier_stddev)
    
# Hint Vector Generation
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C
   
'''
GAIN Consists of 3 Components
- Generator
- Discriminator
- Hint Mechanism
'''   
   
#%% GAIN Architecture   
   
#%% 1. Input Placeholders
# 1.1. Data Vector
X = tf.placeholder(tf.float32, shape = [None, Dim])
# 1.2. Mask Vector 
M = tf.placeholder(tf.float32, shape = [None, Dim])
# 1.3. Hint vector
H = tf.placeholder(tf.float32, shape = [None, Dim])
# 1.4. X with missing values
New_X = tf.placeholder(tf.float32, shape = [None, Dim])

#%% 2. Discriminator
D_W1 = tf.Variable(xavier_init([Dim*2, H_Dim1]))     # Data + Hint as inputs
D_b1 = tf.Variable(tf.zeros(shape = [H_Dim1]))

D_W2 = tf.Variable(xavier_init([H_Dim1, H_Dim2]))
D_b2 = tf.Variable(tf.zeros(shape = [H_Dim2]))

D_W3 = tf.Variable(xavier_init([H_Dim2, Dim]))
D_b3 = tf.Variable(tf.zeros(shape = [Dim]))       # Output is multi-variate

theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]

#%% 3. Generator
G_W1 = tf.Variable(xavier_init([Dim*2, H_Dim1]))     # Data + Mask as inputs (Random Noises are in Missing Components)
G_b1 = tf.Variable(tf.zeros(shape = [H_Dim1]))

G_W2 = tf.Variable(xavier_init([H_Dim1, H_Dim2]))
G_b2 = tf.Variable(tf.zeros(shape = [H_Dim2]))

G_W3 = tf.Variable(xavier_init([H_Dim2, Dim]))
G_b3 = tf.Variable(tf.zeros(shape = [Dim]))

theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]

#%% GAIN Function

#%% 1. Generator
def generator(new_x,m):
    inputs = tf.concat(axis = 1, values = [new_x,m])  # Mask + Data Concatenate
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)   
    G_prob = tf.nn.sigmoid(tf.matmul(G_h2, G_W3) + G_b3) # [0,1] normalized Output
    
    return G_prob
    
#%% 2. Discriminator
def discriminator(new_x, h):
    inputs = tf.concat(axis = 1, values = [new_x,h])  # Hint + Data Concatenate
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)  
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
    D_logit = tf.matmul(D_h2, D_W3) + D_b3
    D_prob = tf.nn.sigmoid(D_logit)  # [0,1] Probability Output
    
    return D_prob

#%% 3. Other functions
# Random sample generator for Z
def sample_Z(m, n):
    return np.random.uniform(0., 0.01, size = [m, n])        

# Mini-batch generation
def sample_idx(m, n):
    A = np.random.permutation(m)
    idx = A[:n]
    return idx

#%% Structure
# Generator
G_sample = generator(New_X,M)

# Combine with original data
Hat_New_X = New_X * M + G_sample * (1-M)

# Discriminator
D_prob = discriminator(Hat_New_X, H)

#%% Loss
D_loss1 = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) + (1-M) * tf.log(1. - D_prob + 1e-8)) 
G_loss1 = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))
MSE_train_loss = tf.reduce_mean((M * New_X - M * G_sample)**2) / tf.reduce_mean(M)

D_loss = D_loss1
G_loss = G_loss1 + alpha * MSE_train_loss 

#%% MSE Performance metric
MSE_test_loss = tf.reduce_mean(((1-M) * X - (1-M)*G_sample)**2) / tf.reduce_mean(1-M)

#%% Solver
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

# Sessions
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#%% Iterations

#%% Start Iterations
for it in tqdm(range(5000)):    
    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
    
    Z_mb = sample_Z(mb_size, Dim) 
    M_mb = trainM[mb_idx,:]  
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    _, D_loss_curr = sess.run([D_solver, D_loss1], feed_dict = {M: M_mb, New_X: New_X_mb, H: H_mb})
    _, G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = sess.run([G_solver, G_loss1, MSE_train_loss, MSE_test_loss],
                                                                       feed_dict = {X: X_mb, M: M_mb, New_X: New_X_mb, H: H_mb})
            
        
    #%% Intermediate Losses
    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('Train_loss: {:.4}'.format(np.sqrt(MSE_train_loss_curr)))
        print('Test_loss: {:.4}'.format(np.sqrt(MSE_test_loss_curr)))
        print()

#%% Final Loss
    
Z_mb = sample_Z(Test_No, Dim) 
M_mb = testM
X_mb = testX

        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
MSE_final, Sample = sess.run([MSE_test_loss, G_sample], feed_dict = {X: testX, M: testM, New_X: New_X_mb})
        
print('\nFinal Test RMSE: ' + str(np.sqrt(MSE_final)))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


  1%|          | 44/5000 [00:00<07:28, 11.05it/s]

Iter: 0
Train_loss: 0.2076
Test_loss: 0.2293



  4%|▎         | 184/5000 [00:00<02:36, 30.79it/s]

Iter: 100
Train_loss: 0.2093
Test_loss: 0.2264



...

Iter: 4800
Train_loss: 0.06785
Test_loss: 0.1245



100%|█████████▉| 4982/5000 [00:11<00:00, 443.91it/s]

Iter: 4900
Train_loss: 0.06521
Test_loss: 0.1375



100%|██████████| 5000/5000 [00:11<00:00, 439.17it/s]


Final Test RMSE: 0.13771257


## Impute missing data for CCPower Plant dataset
Using GAIN technique, impute missing data on CCPower Plant dataset 

In [0]:
# Generate a dataframe with GAIN generated values
    
Z_mb = sample_Z(len(Data), Dim) 
M_mb = Missing
X_mb = Data
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
MSE_final, result = sess.run([MSE_test_loss, G_sample], feed_dict = {X: Data, M: Missing, New_X: New_X_mb})
        
# result is the imputed dataset
result = M_mb * X_mb + (1-M_mb) * result

## Visual comparison of the resulting imputed data
The missing data matrix represent the valid and missing data cells

In [8]:
# orginal data
data_df = pd.DataFrame(Data)
data_df.head(5)

,0,1,2,3,4
0,0.091501,0.271886,0.704281,0.912466,0.849801
1,0.348159,0.319929,0.495422,0.816220,0.595497
2,0.822380,0.736477,0.609998,0.202815,0.155894
3,0.698584,0.392705,0.400891,0.393834,0.253510
4,0.418414,0.343772,0.707993,0.775067,0.511391


In [9]:
# The missing data matrix uses 1 for valid values and 0 for missing NaN values.
missing_df = pd.DataFrame(Missing)
missing_df.head(5)

,0,1,2,3,4
0,1.0,1.0,1.0,0.0,1.0
1,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,0.0,1.0,1.0,0.0,1.0
4,1.0,1.0,1.0,0.0,0.0


In [10]:
result_df = pd.DataFrame(result)
result_df.head(5)

,0,1,2,3,4
0,0.091501,0.271886,0.704281,0.711789,0.849801
1,0.348159,0.319929,0.495422,0.816220,0.595497
2,0.822380,0.736477,0.609998,0.202815,0.155894
3,0.703213,0.392705,0.400891,0.654040,0.253510
4,0.418414,0.343772,0.707993,0.439732,0.582557


## Deep Learning Modeling
We compare the results creating a model using the real dataset and creating another model using the imputed dataset.

We expect to get nearly the same accuracy among models. If the two models give us the same accuracy on a test set, this would be meaning that GAIN method works well generating similar data for filling empty spaces for CCPower Plant dataset.

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers

X_imputed = result_df.iloc[:,0:4]      # features
Y_imputed = result_df.iloc[:,4]        # predict PE variable 

X_test = test_set.iloc[:,0:4]
Y_test = test_set.iloc[:,4]

X_partial = partial_set.iloc[:,0:4]
Y_partial = partial_set.iloc[:,4]

X_full = full_set.iloc[:,0:4]      # features
Y_full = full_set.iloc[:,4]        # predict PE variable 

# define base model
def baseline_model():
	
  # define the keras model
  model = Sequential()
  model.add(Dense(128, input_dim=4, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)))
  model.add(Dropout(0.4))
  model.add(Dense(64, activation='relu',kernel_initializer='normal',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)))
  model.add(Dropout(0.4))
  model.add(Dense(1, kernel_initializer='normal'))
  
  # compile the keras model
  model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])
  return model

Using TensorFlow backend.


## Model for Imputed CCPower Plant Dataset

In [12]:
# create model
model_imputed = baseline_model()
# train model
model_imputed.fit(X_imputed, Y_imputed, epochs=100, verbose=0, validation_split=0.2)
# evaluate model
score = model_imputed.evaluate(X_test, Y_test, verbose=0, batch_size=32)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.










In [13]:
print(model_imputed.metrics_names)
print(score)

['loss', 'mean_squared_error', 'mean_absolute_error']
[0.0616254753193329, 0.051113099956193495, 0.19581719246197704]


## Model for Full Original CCPower Plant Dataset

In [14]:
# create model
model_full = baseline_model()
# train model
model_full.fit(X_full, Y_full, epochs=100, verbose=0, validation_split=0.2)
# evaluate model
score = model_full.evaluate(X_test, Y_test, verbose=0, batch_size=32)

print(model_full.metrics_names)
print(score)

['loss', 'mean_squared_error', 'mean_absolute_error']
[0.06180433492895752, 0.0511047463477854, 0.19602099624166522]
